In [2]:
import pandas as pd
import numpy as np

In [3]:
#Sandwich Game
'''
input:
s: sllipage tolerance
f: transaction fee, 0<=f<1
b: base fee per transaction
X0_reserve: token X reserved in the liquidity pool at time t0
y0_reserve: token Y reserved in the liquidity pool at time t0
X_paid > 0: tokens 𝑋 entering the mempool at time t0;
X_a >0: token X charged when predatory bot first executes a transaction TA1

output:
P_a: bot profits
'''

def sandwich_game(s,f,b,x0_reserve,y0_reserve,X_paid,X_a):
    
    #tokens Y received at time t0
    Y0_receive = (y0_reserve*(1-f)*X_paid)/(x0_reserve+(1-f)*X_paid)
    
    #Sandwich attack occurs
    
    #predatory bot transaction TA1
    Y_a1 = (y0_reserve*(1-f)*X_a)/(x0_reserve+(1-f)*X_a)
    
    #victim transaction at time t1
    x1_reserve = x0_reserve+X_a
    y1_reserve = (x0_reserve*y0_reserve)/(x0_reserve+(1-f)*X_a)
    Victim_receive = (y1_reserve*(1-f)*X_paid)/(x1_reserve+(1-f)*X_paid)
    
    if Victim_receive >= (1-s)*Y0_receive:
        
        #predatory bot exchange TA2 at time t2
        x2_reserve = x0_reserve+X_paid+X_a
        y2_reserve = (x1_reserve*y1_reserve)/(x1_reserve+(1-f)*X_paid)
        Bot_receive = (y2_reserve*(1-f)*Y_a1)/(x1_reserve+(1-f)*Y_a1)
    
        #Bot profits
        P_a = Bot_receive-X_a-2*b
        return P_a
    else:
        return "victim transaction not executed"
    
    

In [4]:
#Adversary Perspective
#Optimal strategy for a predatory bot 
#Finding the optimal attack input X_a that maximizing P_a
'''
input:
s: sllipage tolerance
f: transaction fee, 0<=f<1
b: base fee per transaction
X0_reserve: token X reserved in the liquidity pool at time t0
y0_reserve: token Y reserved in the liquidity pool at time t0
X_paid > 0: tokens 𝑋 entering the mempool at time t0;

output:
X_a: optimal attack input for bots
'''

def predatory_bot(s,f,b,x0_reserve,y0_reserve,X_paid):
    
    #lemma 1: no slippage tolerance, s=1
    Xa_o = ((X_paid*((1-f)**2)*x0_reserve)-f*(x0_reserve**2)*(2-f))/((2-f)*f*x0_reserve-X_paid*f*((1-f)**2))
    +((X_paid**2)*((1-f)**3)*x0_reserve*(x0_reserve-((1-f)**2)*f*(x0_reserve+X_paid)))**(1/2)/((2-f)*f*x0_reserve-X_paid*f*((1-f)**2))
    
    #lemma 2: with silippage tolerance
    n = ((1-f)**2)*(1-s)*(1-s)*((1-f)**4)*(X_paid**2)
    +2*X_paid*((1-f)**2)*x0_reserve*(2-f*(1-s))
    +(4-f*(4-f*(1-s)))*x0_reserve**2
    
    Xa_s = (n/(1-s) - X_paid*(1-f)**3-(2-f)*(1-f)*x0_reserve)/(2*(1-f)**2)
    
    X_a = min(Xa_o,Xa_s)
    
    return X_a
    
    

In [15]:
X_paid = 50000
X_a = predatory_bot(0.01,0.003,0,5000000,0,X_paid)
P = sandwich_game(0.01,0.003, 0, 5000000,0,X_paid,X_a)

In [16]:
P

-3312339.317774605

In [ ]:
#Trader perspective
#Minimizing victim's expected cost
#Finding the optimal slippage tolerance
'''
input:
f: transaction fee, 0<=f<1
b: base fee per transaction
X0_reserve: token X reserved in the liquidity pool at time t0
y0_reserve: token Y reserved in the liquidity pool at time t0
X_paid > 0: tokens 𝑋 entering the mempool at time t0;

output:
s: optimal slippage tolerance
'''
def slippage(f,b,x0_reserve,y0_reserve,X_paid):
    
    #Calculate s_a
    Y0_receive = (y0_reserve*(1-f)*X_paid)/(x0_reserve+(1-f)*X_paid)
    s_a = (2*b)/Y0_receive
    
    #Calculate s_r
    
    # cost of redoing the transaction
    l = 0.25 #the portion of the base fee used for a failed transaction
    m = 0.125 #the potential increase of the base fee in the next block
    c = b*(l+m)
    
    # expected cost of the associated price shift in the pool
    E = *** #calculated with the specific liquidity distribution
    
    # Probability of the transaction failing 
    p = *** #estimated by looking at the recent history of the pool
    
    s_r = ((p)/(1-p))*(c/Y0_receive+E)
    
    s = max(s_a,s_r)
    
    return s

In [ ]:
#Section 5.2: calculated with the specific liquidity distribution.
def expected_cost():
    
    return E

In [ ]:
# Estimate the Probability of the trasaction failing 
def probability_fail():
    
    return p